# **Hand Gesture Detection using YOLOv8**
## Overview
Hand gesture detection is a computer vision task that involves identifying and localizing hand gestures in images or video frames. YOLOv8 (You Only Look Once, Version 8) is a state-of-the-art object detection model that provides high-speed and accurate detection, making it ideal for real-time applications like hand gesture recognition.



## Problem Statement
The goal of this project is to develop a hand gesture detection system using the YOLOv8 (You Only Look Once Version 8) object detection model. The system should be capable of identifying and localizing various hand gestures (e.g., thumbs up, open palm, fist, OK sign) in images or video frames with high accuracy and in real-time. The system must also be robust enough to work in different environmental conditions, with different hand sizes, skin tones, and lighting variations.

In [1]:
import zipfile
import os

In [2]:
if os.path.exists('/kaggle/input/dataset-handgesture'):
    print('dataset already exist')
else:
    !kaggle datasets download -d hand-gesture

dataset already exist


In [4]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as et
from IPython.display import clear_output
import yaml
import shutil
import random
import matplotlib.pyplot as plt
from PIL import Image
plt.rcParams['figure.figsize'] = (10,6)
os.environ['WANDB_DISABLED'] = 'true'
random.seed(42)
os.makedirs('outputs', exist_ok=True)
%matplotlib inline

# **Dataset and Annotations**
## Dataset
The dataset used in this project is coustom datset prepared by myself for this project. This dataset contains rich variety of annoted images depicting hand gestures in various forms, Each image is annoted to indicate Hello, Ok, No, Thanks, Yes.

## Annotation Processing
**1. Data Preprocessing:**
THe inital Step involves parsing TXT annotation files provided with each image in the dataset. These TXT files contain detailed annotations specifying the location and class label of each detected instance within the image.

**2. Dataframe Creation:**
The annotations from TXT files are processed to create a structured dataframe. This dataframe includes entries for each image, listing the filename and associated instances of masks detected. Each instance is characterized by its bounding box coordinates and class label (wearing mask, not wearing mask, or not wearing mask properly).

**3. YOLOv8-Compatible Annotations:**
To facilitate training using YOLOv8, the bounding box coordinates are transformed into YOLO format. This involves converting the coordinates to represent the center of the bounding box relative to the image dimensions, and normalizing these coordinates to range between 0 and 1. Class labels are encoded numerically based on the YOLO model's requirements.

**4. Folder Structure**
The dataset is organized into a YOLOv8-compatible folder structure:
Images: Located in dataset/images, divided into train and validation subfolders.
Labels: Located in dataset/labels, similarly divided into train and validation subfolders.

**5. Config.yaml Creation**
Upon preprocessing the dataset and organizing annotations, a config.yaml file is generated. This configuration file outlines the model settings, including dataset paths and other parameters essential for training the YOLOv8 model.

In [5]:
img_path = '/kaggle/input/dataset-handgesture/Images'
annot_path = '/kaggle/input/dataset-handgesture/Annotations'

In [6]:
df = pd.DataFrame(columns=['file', 'width', 'height', 'instances'])

def get_parsed_txt_data(txt_path, img_width, img_height):
    instances = []
    with open(txt_path, 'r') as file:
        lines = file.readlines()
        
        if not lines:
            print(f"No lines found in {txt_path}")
            return None
        
        for line in lines:
            parts = line.strip().split()
            if len(parts) != 5:
                print(f"Skipping invalid line in {txt_path}: {line.strip()}")
                continue
            class_ = int(parts[0])
            centerx, centery, bbox_width, bbox_height = map(float, parts[1:])
            
            # Convert normalized coordinates to absolute coordinates
            xmin = (centerx - bbox_width / 2) * img_width
            ymin = (centery - bbox_height / 2) * img_height
            xmax = (centerx + bbox_width / 2) * img_width
            ymax = (centery + bbox_height / 2) * img_height
            
            instances.append((class_, [xmin, ymin, xmax, ymax]))
    
    name = os.path.basename(txt_path).replace('.txt', '.jpg')
    return pd.Series([name, img_width, img_height, instances], index=['file', 'width', 'height', 'instances'])

In [7]:
# Main loop to process files
for i in os.listdir(annot_path):
    path = os.path.join(annot_path, i)
    print(f"Processing file: {path}")
    
    # Get image dimensions
    image_name = os.path.basename(path).replace('.txt', '.jpg')
    image_path = os.path.join(img_path, image_name)
    
    try:
        with Image.open(image_path) as img:
            img_width, img_height = img.size
        
        parsed_txt = get_parsed_txt_data(path, img_width, img_height)
        if parsed_txt is not None:
            df = pd.concat([df, parsed_txt.to_frame().T], ignore_index=True)
        else:
            print(f"Skipping file: {path}")
    except FileNotFoundError:
        print(f"Image file not found: {image_path}")
    except IsADirectoryError:
        print(f"Expected a file but found a directory: {image_path}")

Processing file: /kaggle/input/dataset-handgesture/Annotations/Yes.b266516d-741c-11ef-ac62-d03745befa5d.txt
Processing file: /kaggle/input/dataset-handgesture/Annotations/Hello.adff8c13-741c-11ef-8597-d03745befa5d.txt
Processing file: /kaggle/input/dataset-handgesture/Annotations/Thanks.c4cd7cf7-741c-11ef-82cd-d03745befa5d.txt
Processing file: /kaggle/input/dataset-handgesture/Annotations/Hello.ab40c55d-741c-11ef-ba7c-d03745befa5d.txt
Processing file: /kaggle/input/dataset-handgesture/Annotations/Hello.adb45ea1-741c-11ef-83d3-d03745befa5d.txt
Processing file: /kaggle/input/dataset-handgesture/Annotations/Thanks.c427b7d8-741c-11ef-b458-d03745befa5d.txt
Processing file: /kaggle/input/dataset-handgesture/Annotations/Looser.cbeb553a-741c-11ef-aa1b-d03745befa5d.txt
Processing file: /kaggle/input/dataset-handgesture/Annotations/Thanks.c5505d29-741c-11ef-9f18-d03745befa5d.txt
Processing file: /kaggle/input/dataset-handgesture/Annotations/OK.d1abff6b-741c-11ef-91f6-d03745befa5d.txt
Processing 

In [8]:
# Check and process 'instances'
if not df.empty:
    # Apply the function to extract instances (list of bounding boxes)
    instances_df = df['instances'].apply(lambda x: pd.DataFrame(x, columns=['class', 'bbox']) if x else pd.DataFrame(columns=['class', 'bbox']))

    if instances_df.empty:
        print("No instances found.")
    else:
        # For each image, repeat the 'file', 'width', and 'height' the same number of times as the instances for that image
        repeated_data = df[['file', 'width', 'height']].loc[df.index.repeat(instances_df.str.len())].reset_index(drop=True)
        
        # Concatenate the exploded instances DataFrame
        exploded_instances = pd.concat(instances_df.values).reset_index(drop=True)
        
        # Merge the repeated image data with the exploded instances
        final_instances_df = pd.concat([repeated_data, exploded_instances], axis=1)

        # Split the 'bbox' column into separate columns for xmin, ymin, xmax, ymax
        bbox_df = final_instances_df['bbox'].apply(lambda x: pd.Series(x))
        bbox_df.columns = ['xmin', 'ymin', 'xmax', 'ymax']

        # Combine everything into the final DataFrame
        final_df = pd.concat([final_instances_df[['file', 'width', 'height', 'class']], bbox_df], axis=1)

        print("Final DataFrame:")
        print(final_df.head(20))

Final DataFrame:
                                               file width height  class  \
0      Yes.b266516d-741c-11ef-ac62-d03745befa5d.jpg  1920   1080      5   
1    Hello.adff8c13-741c-11ef-8597-d03745befa5d.jpg  1920   1080      0   
2   Thanks.c4cd7cf7-741c-11ef-82cd-d03745befa5d.jpg  1920   1080      4   
3    Hello.ab40c55d-741c-11ef-ba7c-d03745befa5d.jpg  1920   1080      0   
4    Hello.adb45ea1-741c-11ef-83d3-d03745befa5d.jpg  1920   1080      0   
5   Thanks.c427b7d8-741c-11ef-b458-d03745befa5d.jpg  1920   1080      4   
6   Looser.cbeb553a-741c-11ef-aa1b-d03745befa5d.jpg  1920   1080      1   
7   Thanks.c5505d29-741c-11ef-9f18-d03745befa5d.jpg  1920   1080      4   
8       OK.d1abff6b-741c-11ef-91f6-d03745befa5d.jpg  1920   1080      3   
9    Hello.ad0f19ef-741c-11ef-a84a-d03745befa5d.jpg  1920   1080      0   
10   Hello.ad9e74a6-741c-11ef-910c-d03745befa5d.jpg  1920   1080      0   
11  Looser.c9552a56-741c-11ef-a20b-d03745befa5d.jpg  1920   1080      1   
12     Y

In [9]:
def split_data(row):
    class_, bbox = row
    return pd.Series([class_] + bbox)

In [10]:
instances_df = df['instances'].apply(lambda x: pd.DataFrame(x, columns=['class', 'bbox']) if x else pd.DataFrame(columns=['class', 'bbox']))

# Check if instances_df is empty
if instances_df.empty:
    print("No instances found.")
else:
    instances_df = pd.concat([df[['file', 'width', 'height']].loc[df.index.repeat(instances_df.str.len())].reset_index(drop=True), pd.concat(instances_df.values).reset_index(drop=True)], axis=1)

    # Split the 'bbox' column into separate columns
    bbox_df = instances_df['bbox'].apply(lambda x: pd.Series(x))
    bbox_df.columns = ['xmin', 'ymin', 'xmax', 'ymax']

    # Combine all the data into a final DataFrame
    final_df = pd.concat([instances_df[['file', 'width', 'height', 'class']], bbox_df], axis=1)

    print("Final DataFrame:")
    print(final_df.head(20))

Final DataFrame:
                                               file width height  class  \
0      Yes.b266516d-741c-11ef-ac62-d03745befa5d.jpg  1920   1080      5   
1    Hello.adff8c13-741c-11ef-8597-d03745befa5d.jpg  1920   1080      0   
2   Thanks.c4cd7cf7-741c-11ef-82cd-d03745befa5d.jpg  1920   1080      4   
3    Hello.ab40c55d-741c-11ef-ba7c-d03745befa5d.jpg  1920   1080      0   
4    Hello.adb45ea1-741c-11ef-83d3-d03745befa5d.jpg  1920   1080      0   
5   Thanks.c427b7d8-741c-11ef-b458-d03745befa5d.jpg  1920   1080      4   
6   Looser.cbeb553a-741c-11ef-aa1b-d03745befa5d.jpg  1920   1080      1   
7   Thanks.c5505d29-741c-11ef-9f18-d03745befa5d.jpg  1920   1080      4   
8       OK.d1abff6b-741c-11ef-91f6-d03745befa5d.jpg  1920   1080      3   
9    Hello.ad0f19ef-741c-11ef-a84a-d03745befa5d.jpg  1920   1080      0   
10   Hello.ad9e74a6-741c-11ef-910c-d03745befa5d.jpg  1920   1080      0   
11  Looser.c9552a56-741c-11ef-a20b-d03745befa5d.jpg  1920   1080      1   
12     Y

In [11]:
count = 0

if count == 0:
    df = df.explode(column='instances')
    count += 1

# Assuming split_data is a function you've defined to split the 'instances' column
df[['class', 'xmin', 'ymin', 'xmax', 'ymax']] = df['instances'].apply(split_data)
df.head()

,file,width,height,instances,class,xmin,ymin,xmax,ymax
0,Yes.b266516d-741c-11ef-ac62-d03745befa5d.jpg,1920,1080,"(5, [754.9996800000001, 110.00016, 1345.99872,...",5.0,754.99968,110.00016,1345.99872,785.00016
1,Hello.adff8c13-741c-11ef-8597-d03745befa5d.jpg,1920,1080,"(0, [851.00064, 0.9995400000000254, 1258.00031...",0.0,851.00064,0.99954,1258.00032,553.99950
2,Thanks.c4cd7cf7-741c-11ef-82cd-d03745befa5d.jpg,1920,1080,"(4, [658.9987199999999, 375.00030000000004, 11...",4.0,658.99872,375.00030,1117.99968,960.00066
3,Hello.ab40c55d-741c-11ef-ba7c-d03745befa5d.jpg,1920,1080,"(0, [1111.00032, 251.99964000000008, 1468.0012...",0.0,1111.00032,251.99964,1468.00128,843.99948
4,Hello.adb45ea1-741c-11ef-83d3-d03745befa5d.jpg,1920,1080,"(0, [832.0003200000001, 1.000080000000011, 127...",0.0,832.00032,1.00008,1277.00064,513.00000


In [12]:
df.drop(columns=['instances'],inplace=True)
df.head()

,file,width,height,class,xmin,ymin,xmax,ymax
0,Yes.b266516d-741c-11ef-ac62-d03745befa5d.jpg,1920,1080,5.0,754.99968,110.00016,1345.99872,785.00016
1,Hello.adff8c13-741c-11ef-8597-d03745befa5d.jpg,1920,1080,0.0,851.00064,0.99954,1258.00032,553.99950
2,Thanks.c4cd7cf7-741c-11ef-82cd-d03745befa5d.jpg,1920,1080,4.0,658.99872,375.00030,1117.99968,960.00066
3,Hello.ab40c55d-741c-11ef-ba7c-d03745befa5d.jpg,1920,1080,0.0,1111.00032,251.99964,1468.00128,843.99948
4,Hello.adb45ea1-741c-11ef-83d3-d03745befa5d.jpg,1920,1080,0.0,832.00032,1.00008,1277.00064,513.00000


In [13]:
df[['width', 'height', 'xmin', 'ymin', 'xmax', 'ymax']] = df[['width', 'height', 'xmin', 'ymin', 'xmax', 'ymax']].astype(int)

In [14]:
def get_normalized_bounding_boxes(row):
    xmin, xmax = sorted([row['xmin'], row['xmax']])
    ymin, ymax = sorted([row['ymin'], row['ymax']])
    x_center = (xmax + xmin) / 2 / row['width']
    y_center = (ymax + ymin) / 2 / row['height']
    bb_w = (xmax - xmin) / row['width']
    bb_h = (ymax - ymin) / row['height']
    return pd.Series([x_center, y_center, bb_w, bb_h])

In [15]:
df[['xcenter', 'ycenter', 'bb_width', 'bb_height']] = df.apply(get_normalized_bounding_boxes, axis=1)
print("Final DataFrame:")
print(df.head())

Final DataFrame:
                                              file  width  height  class  \
0     Yes.b266516d-741c-11ef-ac62-d03745befa5d.jpg   1920    1080    5.0   
1   Hello.adff8c13-741c-11ef-8597-d03745befa5d.jpg   1920    1080    0.0   
2  Thanks.c4cd7cf7-741c-11ef-82cd-d03745befa5d.jpg   1920    1080    4.0   
3   Hello.ab40c55d-741c-11ef-ba7c-d03745befa5d.jpg   1920    1080    0.0   
4   Hello.adb45ea1-741c-11ef-83d3-d03745befa5d.jpg   1920    1080    0.0   

   xmin  ymin  xmax  ymax   xcenter   ycenter  bb_width  bb_height  
0   754   110  1345   785  0.546615  0.414352  0.307812   0.625000  
1   851     0  1258   553  0.549219  0.256019  0.211979   0.512037  
2   658   375  1117   960  0.462240  0.618056  0.239063   0.541667  
3  1111   251  1468   843  0.671615  0.506481  0.185938   0.548148  
4   832     1  1277   513  0.549219  0.237963  0.231771   0.474074  


In [16]:
# Define the class names
class_names = ['Hello','Looser','No', 'OK', 'Thanks','yes']

# Create the class_map and inverse_map
class_map = {name: idx for idx, name in enumerate(class_names)}
inverse_map = {idx: name for idx, name in enumerate(class_names)}

print("Class Map:", class_map)
print("Inverse Map:", inverse_map)


Class Map: {'Hello': 0, 'Looser': 1, 'No': 2, 'OK': 3, 'Thanks': 4, 'yes': 5}
Inverse Map: {0: 'Hello', 1: 'Looser', 2: 'No', 3: 'OK', 4: 'Thanks', 5: 'yes'}


In [17]:
df['class'] = df['class'].astype(int)

In [18]:
df['enc_class'] = df['class'].apply(lambda x: class_map.get(x, -1))  # Use -1 or any default value for missing keys

# Save the processed DataFrame to a CSV file
df.to_csv('./outputs/processed_data.csv', index=False)

In [19]:
imp_paths = ['dataset',
             'dataset/images',
             'dataset/labels', 
             'dataset/images/train',
             'dataset/images/validation',
             'dataset/labels/train', 
             'dataset/labels/validation']
# Create directories if they don't exist
for i in imp_paths:
    if not os.path.exists(i):
        print(f'Creating {i}')
        os.makedirs(i)
    else:
        print(f"{i} already exists")

Creating dataset
Creating dataset/images
Creating dataset/labels
Creating dataset/images/train
Creating dataset/images/validation
Creating dataset/labels/train
Creating dataset/labels/validation


In [20]:
img_path = '/kaggle/input/dataset-handgesture/Images'
annot_path = '/kaggle/input/dataset-handgesture/Annotations'
train_img = 'dataset/images/train'
val_img = 'dataset/images/validation'
train_label = 'dataset/labels/train'
val_label = 'dataset/labels/validation'

In [21]:
# Shuffle the images and split them into train and validation sets
img_path = '/kaggle/input/dataset-handgesture/Images'  # Update this with the correct path to your images
imgs = os.listdir(img_path)
random.shuffle(imgs)
val_split_size = int(len(imgs) * 0.1)
train = imgs[:len(imgs)-val_split_size]
val = imgs[len(imgs)-val_split_size:]

In [22]:
import os
import shutil

def move_files(files, img_dir, label_dir):
    for file in files:
        base_name = os.path.splitext(file)[0]
        image_path = os.path.join('/kaggle/input/dataset-handgesture/Images', file)
        label_path = os.path.join('/kaggle/input/dataset-handgesture/Annotations', base_name + '.txt')
        
        # Copy image file
        shutil.copy(image_path, os.path.join(img_dir, file))
        
        # Copy annotation file if it exists
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(label_dir, base_name + '.txt'))

# Move training files
move_files(train, train_img, train_label)

# Move validation files
move_files(val, val_img, val_label)

print("Dataset split and moved successfully!")

Dataset split and moved successfully!


In [23]:
data = {
    'path': os.path.abspath('dataset'),
    'train': os.path.abspath('dataset/images/train'),
    'val': os.path.abspath('dataset/images/validation'),
    'nc': len(class_map),  # Number of classes
    'names': inverse_map   # Class names
}

# Write the data dictionary to a YAML file
with open('./config.yaml', 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

print("Configuration YAML file created successfully.")


Configuration YAML file created successfully.


# Introduction to YOLOv8
YOLOv8 is the latest iteration in the YOLO (You Only Look Once) family, known for its real-time performance and accuracy.
It builds on its predecessors, offering improvements in efficiency, accuracy, and support for custom object detection tasks, such as hand gesture detection.
YOLOv8 is designed to work well on various devices, from high-end GPUs to mobile platforms, which makes it suitable for deployment in real-time applications like gesture-controlled systems.


In [24]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.7/872.7 kB 30.7 MB/s eta 0:00:00


In [25]:
import ultralytics 
from ultralytics import YOLO
model =  None
to_train = False


In [26]:
!pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 73.2 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.8
    Uninstalling widgetsnbextension-3.6.8:
      Successfully uninstalled widgetsnbextension-3.6.8
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.0.11
    Uninstalling jupyterlab_widgets-3.0.11:
      Successfully uninstalled jupyterlab_widgets-3.0.11
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0

In [27]:
import os
train_label_dir = '/kaggle/working/dataset/labels/train'
label_files = os.listdir(train_label_dir)
if len(label_files) == 0:
    print("No label files found in the train label directory.")
else:
    print(f"{len(label_files)} label files found.")


416 label files found.


In [ ]:
model = YOLO('yolov8l.pt')
model.train(data='/kaggle/working/config.yaml', epochs=200)

Ultralytics YOLOv8.2.94 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/kaggle/working/config.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

train: Scanning /kaggle/working/dataset/labels/train.cache... 416 images, 0 backgrounds, 0 corrupt: 100%|██████████| 416/416 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/dataset/labels/validation.cache... 46 images, 0 backgrounds, 0 corrupt: 100%|██████████| 46/46 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to runs/detect/train3
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         46         46      0.994          1      0.995      0.617



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    139/200      10.6G       1.23     0.4648      1.434         33        640: 100%|██████████| 26/26 [00:23<00:00,  1.09it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.21it/s]


                   all         46         46      0.993          1      0.995      0.621

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    140/200      10.8G       1.16     0.4985      1.444         37        640:   8%|▊         | 2/26 [00:02<00:22,  1.07it/s]

# Evaluation

## Evaluation Metrics

Model performance is rigorously evaluated using metrics such as precision, recall, and mean average precision (mAP). Precision measures the accuracy of the model’s positive predictions, while recall assesses its ability to identify all relevant instances. The mAP provides a comprehensive measure of the model’s overall accuracy across various classes and intersection over union (IoU) thresholds. Evaluation on validation datasets ensures that the model consistently performs well, maintaining high accuracy and reliability in diverse real-world scenarios.



In [ ]:
import pandas as pd

# Evaluate the model
metrics = model.val(data='config.yaml', verbose=False)

# Create evaluation data dictionary
eval_data = {
    'metrics': ['mAP(50-95)', 'mAP(50)', 'mAP(75)'],
    'score': [metrics.box.map, metrics.box.map50, metrics.box.map75]
}

# Create mAP over classes dictionary
mAP_over_cls = {
    'class': ['Hello','Looser','No', 'OK','Thanks','Yes' ],
    'mAP': metrics.box.maps,
    'precision': metrics.box.p,
    'recall': metrics.box.r
}

# Convert dictionaries to DataFrames
eval_df = pd.DataFrame(eval_data)
mAP_cls = pd.DataFrame(mAP_over_cls)

# Display the DataFrames
print(eval_df)
print(mAP_cls)


In [ ]:
# Print lengths of each field in mAP_over_cls
print(f"Classes: {len(mAP_over_cls['class'])}")
print(f"mAP: {len(mAP_over_cls['mAP'])}")
print(f"Precision: {len(mAP_over_cls['precision'])}")
print(f"Recall: {len(mAP_over_cls['recall'])}")

# Check if the lengths match before creating DataFrame
if len(mAP_over_cls['class']) == len(mAP_over_cls['mAP']) == len(mAP_over_cls['precision']) == len(mAP_over_cls['recall']):
    mAP_cls = pd.DataFrame(mAP_over_cls)
    print(mAP_cls)
else:
    print("Mismatch in array lengths. Please verify the data.")


In [ ]:
eval_df.plot(kind='line',x='metrics',y='score')

In [ ]:
barWidth = 0.25
r1 = np.arange(len(mAP_cls['class']))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
plt.bar(r1, mAP_cls['mAP'], color='b', width=barWidth, edgecolor='grey', label='mAP')
plt.bar(r2, mAP_cls['precision'], color='g', width=barWidth, edgecolor='grey', label='precision')
plt.bar(r3, mAP_cls['recall'], color='r', width=barWidth, edgecolor='grey', label='recall')
plt.xlabel('class', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(mAP_cls['class']))], mAP_cls['class'])
plt.legend()
plt.title("Evaluation of each category")
plt.show()

In [ ]:
import matplotlib.image as mpimg

# Load and display the image
f1_curve = mpimg.imread('/kaggle/working/runs/detect/train/F1_curve.png')
plt.imshow(f1_curve)
plt.axis('off')  # Hide axes
plt.title('F1 Curve')
plt.show()

In [ ]:

P_curve = mpimg.imread('/kaggle/working/runs/detect/train/P_curve.png')
plt.imshow(P_curve)
plt.axis('off')  # Hide axes
plt.title('P Curve')
plt.show()

In [ ]:

PR_curve = mpimg.imread('/kaggle/working/runs/detect/train/PR_curve.png')
plt.imshow(PR_curve)
plt.axis('off')  # Hide axes
plt.title('PR Curve')
plt.show()

In [ ]:

Confusion_matrix = mpimg.imread('/kaggle/working/runs/detect/train/confusion_matrix_normalized.png')
plt.imshow(Confusion_matrix)
plt.axis('off')  # Hide axes
plt.title('Confusion_matrix')
plt.show()

In [ ]:
import cv2
from time import sleep

In [ ]:
def get_boxed_img(img, result, conf_thres=0.65):
    """
    Returns image with bounding boxes.
    
    Args:
        img : Image on which to draw bounding boxes.
        result : Output of YOLO prediction.
        conf_thres : Confidence threshold to follow. default 0.65
    """
    boxes = result.boxes.xyxy.tolist()
    class_id = result.boxes.cls.tolist()
    name = result.names
    confidence = result.boxes.conf.tolist()
    
    for box, cls, cnf in zip(boxes, class_id, confidence):
        if cnf < conf_thres:
            continue
        
        color = (0, 0, 0)
        if cls == 0:
            color = (0, 255, 0)  # Green
        elif cls == 2:
            color = (255, 0, 0)  # Red
        else:
            color = (0, 0, 255)  # Blue
        
        x1, y1, x2, y2 = map(int, box)
        
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        
        text = f"{name[cls]} {cnf:.2f}"
        
        font_scale = max(min((x2 - x1) / 100, (y2 - y1) / 100), 0.5)
        font_thickness = max(int(font_scale * 2), 1)
        
        (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
        text_x = x1 + (x2 - x1) // 2 - text_width // 2
        text_y = max(y1 - 10, text_height + 5)
        
        cv2.rectangle(img, (text_x - 5, text_y - text_height - 5), (text_x + text_width + 5, text_y + 5), color, -1)
        
        cv2.putText(img, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness, cv2.LINE_AA)
    
    return img

In [ ]:
for i in os.listdir('./dataset/images/validation')[:10]:
    clear_output(wait=True)
    path = os.path.join('./dataset/images/validation',i)
    plt.figure(figsize=(12,8))
    plt.subplot(1,2,1)
    img = cv2.imread(path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.axis('off')
    plt.tight_layout()
    plt.subplot(1,2,2)
    result = model.predict(path,verbose=False)[0]
    imgp = get_boxed_img(img,result)
    plt.imshow(imgp)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    sleep(2)

In [ ]:
# Video parameters
video_file = './outputs/output.mp4'  # Change to .mp4
image_save_path = './outputs/temp.jpeg'
frame_rate = 30  
duration_per_image = 2  # In seconds
height, width, channels = 800, 800, 3
bit_rate = 5000

try:
    # Use 'mp4v' codec instead of VP90 for MP4 format
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(video_file, fourcc, frame_rate, (width, height))
    assert writer.isOpened()
    print("Supported")
except:
    print("Not supported")

# Loop through images and write plots to video
image_folder = './dataset/images/validation'
for i in os.listdir(image_folder):
    path = os.path.join(image_folder, i)
    
    # Display original and processed image side by side
    plt.figure(figsize=(width // 100, height // 100))

    # Plot original image
    plt.subplot(1, 2, 1)
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.axis('off')
    plt.tight_layout()

    # Plot processed image (placeholder for your model prediction)
    plt.subplot(1, 2, 2)
    # Replace this with actual processing code
    imgp = img  # Placeholder: replace with model.predict and get_boxed_img
    plt.imshow(imgp)
    plt.axis('off')
    plt.tight_layout()

    # Save the plot to an image file
    plt.savefig(image_save_path, format='jpeg')
    plt.close()

    img = cv2.imread(image_save_path)

    # Write the same frame for the duration of `duration_per_image` seconds
    for _ in range(frame_rate * duration_per_image):
        writer.write(img)

writer.release()
os.remove(image_save_path)
print(f"Video saved successfully: {video_file}")

# Results
The trained YOLOv8 model achieved notable accuracy in detecting hand gestures and classifying various hand signs. Its high performance across different gesture categories and environments highlights its effectiveness in real-time applications. This makes it well-suited for integration into systems for human-computer interaction, gesture-based control, and other applications requiring reliable gesture recognition.

# Conclusion
YOLOv8 provides a powerful and efficient framework for hand gesture detection, with the ability to deliver accurate results in real-time. By leveraging the model’s fast inference speed and high precision, it can be used in a wide range of applications such as gesture-based control systems, AR/VR, and sign language interpretation.